In [2]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import tensorflow as tf
import logging
import gc

In [3]:
class MaskDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Real-Time Mask Detection")
        self.root.geometry("800x600")
        try:
            icon_image = Image.open("logo.png")
            icon_photo = ImageTk.PhotoImage(icon_image)
            self.root.iconphoto(True, icon_photo)
            self.icon_photo = icon_photo
        except Exception as e:
            logging.error(f"Error loading icon: {str(e)}")
            messagebox.showwarning("Warning", "Unable to load application icon.")
        self.model = tf.keras.models.load_model("mask_detection_model.keras")
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Cannot access webcam")
            self.root.quit()
        self.label = tk.Label(self.root)
        self.label.pack(pady=10)
        self.result_label = tk.Label(self.root, text="Prediction: None", font=("Arial", 14))
        self.result_label.pack(pady=10)
        self.btn_capture = tk.Button(self.root, text="Capture & Predict", command=self.capture_and_predict, font=("Arial", 12), bg="#000080", fg="white")
        self.btn_capture.pack(pady=5)
        self.btn_exit = tk.Button(self.root, text="Exit", command=self.exit_app, font=("Arial", 12), bg="#000080", fg="white")
        self.btn_exit.pack(pady=5)
        self.root.bind('q', lambda event: self.exit_app())
        self.running = True
        self.update_frame()

    def update_frame(self):
        if self.running:
            ret, frame = self.cap.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_resized = cv2.resize(frame_rgb, (640, 480))
                img = Image.fromarray(frame_resized)
                imgtk = ImageTk.PhotoImage(image=img)
                self.label.imgtk = imgtk
                self.label.configure(image=imgtk)
            self.root.after(10, self.update_frame)

    def capture_and_predict(self):
        ret, frame = self.cap.read()
        if ret:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128, 128))
            img = np.array(img) / 255.0
            img = np.expand_dims(img, axis=0)
            prediction = self.model.predict(img)
            class_idx = np.argmax(prediction[0])
            confidence = prediction[0][class_idx]
            result = "With Mask" if class_idx == 1 else "Without Mask"
            self.result_label.config(text=f"Prediction: {result} ({confidence:.2%})")

    def exit_app(self):
        self.running = False
        if self.cap is not None:
            self.cap.release()
        if hasattr(self.label, 'imgtk'):
            self.label.imgtk = None
        self.label.destroy()
        self.result_label.destroy()
        self.btn_capture.destroy()
        self.btn_exit.destroy()
        gc.collect()
        self.root.quit()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = MaskDetectionApp(root)
    root.mainloop()

1/1 [==============================] - 0s 23ms/step
